# Generated

In [ ]:
import numpy as np
from scipy.spatial.distance import pdist, cdist
from itertools import combinations
import matplotlib.pyplot as plt

# Wczytanie danych

# ŚCIEŻKA DOSTĘPU DO WYGENEROWANYCH DANYCH

print("Wczytywanie danych...")
images = np.load('/generated_samples.npy')  # Shape: (1000, height, width, 3)
labels = np.load('/generated_labels.npy')  # Shape: (1000,)

print(f"Kształt obrazów: {images.shape}")
print(f"Kształt etykiet: {labels.shape}")
print(f"Liczba unikalnych klas: {len(np.unique(labels))}")
print(f"Klasy: {np.unique(labels)}")

# Spłaszczenie obrazów dla każdego kanału
print("\nSpłaszczanie kanałów...")
height, width = images.shape[1], images.shape[2]
n_pixels = height * width

# Spłaszczenie każdego kanału osobno
r_channel = images[:, 0, :, :].reshape(1000, -1)  # Kanał R
g_channel = images[:, 1, :, :].reshape(1000, -1)  # Kanał G
b_channel = images[:, 2, :, :].reshape(1000, -1)  # Kanał B

print(f"Kształt spłaszczonego kanału: {r_channel.shape}")

def calculate_intra_class_distances(channel_data, labels, channel_name):
    """Oblicza średnią odległość i odchylenie standardowe wewnątrz każdej klasy"""
    print(f"\n=== Analiza kanału {channel_name} ===")

    unique_classes = np.unique(labels)
    intra_class_stats = {}

    for class_id in unique_classes:
        # Pobierz obrazy z danej klasy
        class_mask = labels == class_id
        class_images = channel_data[class_mask]

        if len(class_images) > 1:
            # Oblicz odległości euklidesowe między wszystkimi parami w klasie
            distances = pdist(class_images, metric='euclidean')

            mean_dist = np.mean(distances)
            std_dist = np.std(distances)

            intra_class_stats[class_id] = {
                'mean_distance': mean_dist,
                'std_distance': std_dist,
                'n_samples': len(class_images),
                'n_pairs': len(distances)
            }

            print(f"Klasa {class_id}: {len(class_images)} obrazów")
            print(f"  Średnia odległość: {mean_dist:.2f}")
            print(f"  Odchylenie standardowe: {std_dist:.2f}")
            print(f"  Liczba par: {len(distances)}")
        else:
            print(f"Klasa {class_id}: tylko {len(class_images)} obraz(ów) - pomijam")

    return intra_class_stats

def calculate_inter_class_distances(channel_data, labels, channel_name):
    """Oblicza średnią odległość między wszystkimi parami klas"""
    print(f"\n=== Odległości między klasami - kanał {channel_name} ===")

    unique_classes = np.unique(labels)
    inter_class_stats = {}

    # Oblicz średnie obrazy dla każdej klasy
    class_centroids = {}
    for class_id in unique_classes:
        class_mask = labels == class_id
        class_images = channel_data[class_mask]
        class_centroids[class_id] = np.mean(class_images, axis=0)

    # Oblicz odległości między wszystkimi parami klas
    for class1, class2 in combinations(unique_classes, 2):
        # Pobierz wszystkie obrazy z obu klas
        class1_images = channel_data[labels == class1]
        class2_images = channel_data[labels == class2]

        # Oblicz odległości między wszystkimi parami obrazów z różnych klas
        cross_distances = cdist(class1_images, class2_images, metric='euclidean')
        mean_cross_dist = np.mean(cross_distances)

        # Odległość między centroidami
        centroid_dist = np.linalg.norm(class_centroids[class1] - class_centroids[class2])

        inter_class_stats[(class1, class2)] = {
            'mean_distance': mean_cross_dist,
            'centroid_distance': centroid_dist,
            'n_pairs': cross_distances.size
        }

        print(f"Klasy {class1} - {class2}:")
        print(f"  Średnia odległość między obrazami: {mean_cross_dist:.2f}")
        print(f"  Odległość między centroidami: {centroid_dist:.2f}")
        print(f"  Liczba par: {cross_distances.size}")

    return inter_class_stats

# Analiza dla każdego kanału
channels = [
    (r_channel, 'R'),
    (g_channel, 'G'),
    (b_channel, 'B')
]

all_intra_stats = {}
all_inter_stats = {}

for channel_data, channel_name in channels:
    print(f"\n{'='*50}")
    print(f"ANALIZA KANAŁU {channel_name}")
    print(f"{'='*50}")

    # Analiza wewnątrzklasowa
    intra_stats = calculate_intra_class_distances(channel_data, labels, channel_name)
    all_intra_stats[channel_name] = intra_stats

    # Analiza międzyklasowa
    inter_stats = calculate_inter_class_distances(channel_data, labels, channel_name)
    all_inter_stats[channel_name] = inter_stats

# Podsumowanie
print(f"\n{'='*50}")
print("PODSUMOWANIE")
print(f"{'='*50}")

for channel_name in ['R', 'G', 'B']:
    print(f"\nKanał {channel_name}:")

    # Średnia odległość wewnątrzklasowa
    intra_means = [stats['mean_distance'] for stats in all_intra_stats[channel_name].values()]
    if intra_means:
        print(f"  Średnia odległość wewnątrzklasowa: {np.mean(intra_means):.2f} ± {np.std(intra_means):.2f}")

    # Średnia odległość międzyklasowa
    inter_means = [stats['mean_distance'] for stats in all_inter_stats[channel_name].values()]
    if inter_means:
        print(f"  Średnia odległość międzyklasowa: {np.mean(inter_means):.2f} ± {np.std(inter_means):.2f}")

print("\nAnaliza zakończona!")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

  # Wizualizacja wyników
def plot_distance_comparison1():
  # Set high-quality scientific style
  plt.rcParams.update({
      'font.size': 12,
      'font.family': 'serif',
      'font.serif': ['Times New Roman', 'Computer Modern Roman'],
      'axes.linewidth': 1.2,
      'axes.spines.left': True,
      'axes.spines.bottom': True,
      'axes.spines.top': False,
      'axes.spines.right': False,
      'xtick.major.size': 6,
      'xtick.minor.size': 3,
      'ytick.major.size': 6,
      'ytick.minor.size': 3,
      'legend.frameon': True,
      'legend.fancybox': False,
      'legend.edgecolor': 'black',
      'legend.facecolor': 'white',
      'legend.framealpha': 0.9,
      'figure.dpi': 300,
      'savefig.dpi': 300,
      'savefig.bbox': 'tight',
      'savefig.pad_inches': 0.1
  })

  # Set seaborn style for scientific publications
  sns.set_style("whitegrid", {
      'axes.grid': True,
      'axes.edgecolor': 'black',
      'grid.color': 'gray',
      'grid.alpha': 0.4,
      'grid.linewidth': 0.8
  })

  # Set color palette - using colorblind-friendly colors
  colors = ['red', 'green', 'blue']

  # Create figure with higher DPI and better proportions
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6), dpi=300)
  ax1.set_ylim(15, 95)
  ax2.set_ylim(15, 95)

  channels = ['R', 'G', 'B']

  # Prepare data for seaborn (convert to long format)
  # Intra-class distances
  intra_df_list = []
  for i, channel in enumerate(channels):
      means = [stats['mean_distance'] for stats in all_intra_stats[channel].values()]
      for mean_val in means:
          intra_df_list.append({'Channel': channel, 'Distance': mean_val, 'Type': 'Intra-class'})

  intra_df = pd.DataFrame(intra_df_list)

  # Inter-class distances
  inter_df_list = []
  for i, channel in enumerate(channels):
      means = [stats['mean_distance'] for stats in all_inter_stats[channel].values()]
      for mean_val in means:
          inter_df_list.append({'Channel': channel, 'Distance': mean_val, 'Type': 'Inter-class'})

  inter_df = pd.DataFrame(inter_df_list)

  # Plot 1: Intra-class distances with seaborn
  sns.boxplot(
    data=intra_df,
    x='Channel',
    y='Distance',
    hue='Channel',
    ax=ax1,
    palette=colors,
    linewidth=1.2,
    fliersize=4,
    legend=False)
  ax1.set_title('Intra-class Distances - Generated', fontsize=14, pad=10)
  ax1.set_ylabel('Euclidean Distance', fontsize=12, fontweight='bold')
  ax1.set_xlabel('Color Channel', fontsize=12, fontweight='bold')
  ax1.tick_params(axis='both', which='major', labelsize=10)

  # Add statistical annotations (mean values)
  for i, channel in enumerate(channels):
      means = [stats['mean_distance'] for stats in all_intra_stats[channel].values()]
      mean_val = np.mean(means)
      ax1.text(i, ax1.get_ylim()[1] * 0.95, f'mean = {mean_val:.1f}',
              ha='center', va='top', fontsize=9, fontweight='bold',
              bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

  # Plot 2: Inter-class distances with seaborn
  sns.boxplot(
    data=inter_df,
    x='Channel',
    y='Distance',
    hue='Channel',
    ax=ax2,
    palette=colors,
    linewidth=1.2,
    fliersize=4,
    legend=False)
  ax2.set_title('Inter-class Distances - Generated', fontsize=14, pad=10)
  ax2.set_ylabel('Euclidean Distance', fontsize=12, fontweight='bold')
  ax2.set_xlabel('Color Channel', fontsize=12, fontweight='bold')
  ax2.tick_params(axis='both', which='major', labelsize=10)

  # Add statistical annotations (mean values)
  for i, channel in enumerate(channels):
      means = [stats['mean_distance'] for stats in all_inter_stats[channel].values()]
      mean_val = np.mean(means)
      ax2.text(i, ax2.get_ylim()[1] * 0.95, f'mean = {mean_val:.1f}',
              ha='center', va='top', fontsize=9, fontweight='bold',
              bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

  # Improve layout
  plt.tight_layout(rect=[0, 0.03, 1, 0.93])

  # Add subtle background color for better contrast
  fig.patch.set_facecolor('white')

  # Optional: Add grid customization for better readability
  for ax in [ax1, ax2]:
      ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.8)
      ax.set_axisbelow(True)

  plt.show()

  # Uruchom wizualizację
plot_distance_comparison1()

In [ ]:
def plot_intra_class_spiderplot(all_intra_stats):
    import matplotlib.pyplot as plt
    import numpy as np

    channels = ['R', 'G', 'B']
    class_names = ['basophil', 'eosinophil', 'erythroblast', 'granulocyte',
                   'lymphocyte', 'monocyte', 'neutrophil', 'platelet']
    colors = ['#E53E3E', '#38A169', '#3182CE']  # R, G, B

    angles = np.linspace(0, 2 * np.pi, len(class_names), endpoint=False).tolist()
    angles += angles[:1]  # close loop

    plt.rcParams['font.family'] = 'sans-serif'

    fig, ax = plt.subplots(figsize=(10, 8), subplot_kw=dict(polar=True))

    for i, channel in enumerate(channels):
        stats = all_intra_stats[channel]
        means = [stats.get(cls_id, {'mean_distance': 0})['mean_distance'] for cls_id in range(len(class_names))]
        means += means[:1]  # close the loop

        ax.plot(angles, means, label=f'{channel}', color=colors[i], linewidth=2)
        ax.fill(angles, means, color=colors[i], alpha=0.15)

    # Custom label placement with offset and angle
    label_radius = ax.get_rmax() * 1.13  # Push labels out a bit beyond the outermost grid
    for angle, label in zip(angles[:-1], class_names):
        ax.text(angle, label_radius, label,
                fontsize=10, ha='center', va='center',
                rotation=25, rotation_mode='anchor')

    ax.set_title('Mean Intra-class Euclidean Distances - RGB Channel Analysis\nGenerated',
                 pad=50, fontsize=14)

    ax.set_xticks(angles[:-1])  # Keep angles for radial lines
    ax.set_xticklabels([])  # But remove their labels
    ax.set_rlabel_position(0)
    ax.xaxis.grid(True, color='gray', linestyle='--', linewidth=0.5)  # Radial lines
    ax.yaxis.grid(True)  # Circular gridlines

    # Legend in bottom-right corner
    ax.legend(loc='lower right', bbox_to_anchor=(1.25, -0.1), fontsize=10)

    plt.tight_layout()
    plt.show()

# ZMIENNA all_intra_stats TO SŁOWNIK STWORZONY W KODZIE W KOMÓRCE WYŻEJ PODCZAS OBLICZANIA ODLEGŁOŚCI

plot_intra_class_spiderplot(all_intra_stats)

# BloodMNIST

In [ ]:
!pip install medmnist

In [ ]:
import numpy as np
from scipy.spatial.distance import pdist, cdist
from itertools import combinations
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import medmnist
from medmnist import BloodMNIST
import torchvision.transforms as transforms

# Ustawienia
SAMPLES_PER_CLASS = 128
IMAGE_SIZE = 128
np.random.seed(42)  # dla reprodukowalności

print("Wczytywanie danych BloodMNIST...")

# Wczytanie danych BloodMNIST z rozdzielczością 128x128
data_flag = 'bloodmnist'
download = True

# Transformacje danych
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])  # Normalizacja do [-1, 1]
])

# Ładowanie danych BloodMNIST
train_dataset = BloodMNIST(split='train', transform=data_transforms, download=True, as_rgb=True, size=IMAGE_SIZE)
val_dataset = BloodMNIST(split='val', transform=data_transforms, download=True, as_rgb=True, size=IMAGE_SIZE)
test_dataset = BloodMNIST(split='test', transform=data_transforms, download=True, as_rgb=True, size=IMAGE_SIZE)

In [ ]:
# Kombinowanie wszystkich splitów
all_images = []
all_labels = []

for dataset in [train_dataset, val_dataset, test_dataset]:
    for i in range(len(dataset)):
        img, label = dataset[i]
        all_images.append(np.array(img))
        all_labels.append(label)

all_images = np.array(all_images)
all_labels = np.array(all_labels).flatten()

print(f"Całkowita liczba obrazów: {len(all_images)}")
print(f"Kształt obrazów: {all_images.shape}")
print(f"Unikalne klasy: {np.unique(all_labels)}")
print(f"Liczba klas: {len(np.unique(all_labels))}")

# Sprawdzenie liczby próbek w każdej klasie
unique_classes = np.unique(all_labels)
for class_id in unique_classes:
    count = np.sum(all_labels == class_id)
    print(f"Klasa {class_id}: {count} obrazów")

# Losowanie po 128 próbek z każdej klasy
print(f"\nLosowanie po {SAMPLES_PER_CLASS} próbek z każdej klasy...")

selected_images = []
selected_labels = []

for class_id in unique_classes:
    class_mask = all_labels == class_id
    class_images = all_images[class_mask]
    class_labels = all_labels[class_mask]

    if len(class_images) >= SAMPLES_PER_CLASS:
        # Losowe wybieranie próbek
        indices = np.random.choice(len(class_images), SAMPLES_PER_CLASS, replace=False)
        selected_images.append(class_images[indices])
        selected_labels.append(class_labels[indices])
        print(f"Klasa {class_id}: wybrano {SAMPLES_PER_CLASS} z {len(class_images)} dostępnych")
    else:
        # Jeśli mniej niż 128 próbek, weź wszystkie i uzupełnij duplikatami
        n_available = len(class_images)
        n_duplicates = SAMPLES_PER_CLASS - n_available

        # Wszystkie dostępne próbki
        selected_class_images = class_images.copy()
        selected_class_labels = class_labels.copy()

        # Dodaj losowe duplikaty
        if n_duplicates > 0:
            duplicate_indices = np.random.choice(n_available, n_duplicates, replace=True)
            duplicate_images = class_images[duplicate_indices]
            duplicate_labels = class_labels[duplicate_indices]

            selected_class_images = np.concatenate([selected_class_images, duplicate_images])
            selected_class_labels = np.concatenate([selected_class_labels, duplicate_labels])

        selected_images.append(selected_class_images)
        selected_labels.append(selected_class_labels)
        print(f"Klasa {class_id}: wybrano {n_available} oryginalnych + {n_duplicates} duplikatów")

# Łączenie wszystkich wybranych próbek
images = np.concatenate(selected_images, axis=0)
labels = np.concatenate(selected_labels, axis=0)

print(f"\nFinalne dane:")
print(f"Kształt obrazów: {images.shape}")
print(f"Kształt etykiet: {labels.shape}")
print(f"Liczba obrazów na klasę:")
for class_id in unique_classes:
    count = np.sum(labels == class_id)
    print(f"  Klasa {class_id}: {count} obrazów")

# Sprawdzenie czy obrazy są RGB czy grayscale
if len(images.shape) == 3:
    # Grayscale - konwertuj do RGB przez duplikowanie kanału
    print("Obrazy w skali szarości - konwersja do RGB...")
    images = np.stack([images] * 3, axis=-1)
    print(f"Nowy kształt po konwersji: {images.shape}")

# Spłaszczenie obrazów dla każdego kanału
print("\nSpłaszczanie kanałów...")
height, width = images.shape[1], images.shape[2]
n_pixels = height * width
total_samples = len(images)

# Spłaszczenie każdego kanału osobno
r_channel = images[:, 0, :, :].reshape(total_samples, -1)  # Kanał R
g_channel = images[:, 1, :, :].reshape(total_samples, -1)  # Kanał G
b_channel = images[:, 2, :, :].reshape(total_samples, -1)  # Kanał B

print(f"Kształt spłaszczonego kanału: {r_channel.shape}")

def calculate_intra_class_distances(channel_data, labels, channel_name):
    """Oblicza średnią odległość i odchylenie standardowe wewnątrz każdej klasy"""
    print(f"\n=== Analiza kanału {channel_name} ===")

    unique_classes = np.unique(labels)
    intra_class_stats = {}

    for class_id in unique_classes:
        # Pobierz obrazy z danej klasy
        class_mask = labels == class_id
        class_images = channel_data[class_mask]

        if len(class_images) > 1:
            # Ze względu na dużą liczbę próbek, używamy próbkowania dla obliczenia odległości
            max_pairs = 5000  # Maksymalna liczba par do obliczenia (aby uniknąć problemów z pamięcią)
            n_samples = len(class_images)
            n_possible_pairs = n_samples * (n_samples - 1) // 2

            if n_possible_pairs <= max_pairs:
                # Oblicz wszystkie odległości
                distances = pdist(class_images, metric='euclidean')
            else:
                # Losowe próbkowanie par
                print(f"    Próbkowanie {max_pairs} par z {n_possible_pairs} możliwych...")
                sample_indices = np.random.choice(len(class_images),
                                                min(200, len(class_images)),
                                                replace=False)
                sample_images = class_images[sample_indices]
                distances = pdist(sample_images, metric='euclidean')

            mean_dist = np.mean(distances)
            std_dist = np.std(distances)

            intra_class_stats[class_id] = {
                'mean_distance': mean_dist,
                'std_distance': std_dist,
                'n_samples': len(class_images),
                'n_pairs': len(distances)
            }

            print(f"Klasa {class_id}: {len(class_images)} obrazów")
            print(f"  Średnia odległość: {mean_dist:.2f}")
            print(f"  Odchylenie standardowe: {std_dist:.2f}")
            print(f"  Liczba par (obliczonych): {len(distances)}")
        else:
            print(f"Klasa {class_id}: tylko {len(class_images)} obraz(ów) - pomijam")

    return intra_class_stats

def calculate_inter_class_distances(channel_data, labels, channel_name):
    """Oblicza średnią odległość między wszystkimi parami klas"""
    print(f"\n=== Odległości między klasami - kanał {channel_name} ===")

    unique_classes = np.unique(labels)
    inter_class_stats = {}

    # Oblicz średnie obrazy dla każdej klasy (centroidy)
    class_centroids = {}
    for class_id in unique_classes:
        class_mask = labels == class_id
        class_images = channel_data[class_mask]
        class_centroids[class_id] = np.mean(class_images, axis=0)

    # Oblicz odległości między wszystkimi parami klas
    for class1, class2 in combinations(unique_classes, 2):
        # Pobierz obrazy z obu klas
        class1_images = channel_data[labels == class1]
        class2_images = channel_data[labels == class2]

        # Ze względu na dużą liczbę próbek, użyj próbkowania
        max_samples_per_class = 50
        if len(class1_images) > max_samples_per_class:
            idx1 = np.random.choice(len(class1_images), max_samples_per_class, replace=False)
            class1_sample = class1_images[idx1]
        else:
            class1_sample = class1_images

        if len(class2_images) > max_samples_per_class:
            idx2 = np.random.choice(len(class2_images), max_samples_per_class, replace=False)
            class2_sample = class2_images[idx2]
        else:
            class2_sample = class2_images

        # Oblicz odległości między próbkami z różnych klas
        cross_distances = cdist(class1_sample, class2_sample, metric='euclidean')
        mean_cross_dist = np.mean(cross_distances)

        # Odległość między centroidami
        centroid_dist = np.linalg.norm(class_centroids[class1] - class_centroids[class2])

        inter_class_stats[(class1, class2)] = {
            'mean_distance': mean_cross_dist,
            'centroid_distance': centroid_dist,
            'n_pairs': cross_distances.size
        }

        print(f"Klasy {class1} - {class2}:")
        print(f"  Średnia odległość między obrazami: {mean_cross_dist:.2f}")
        print(f"  Odległość między centroidami: {centroid_dist:.2f}")
        print(f"  Liczba par (próbkowanych): {cross_distances.size}")

    return inter_class_stats

# Analiza dla każdego kanału
channels = [
    (r_channel, 'R'),
    (g_channel, 'G'),
    (b_channel, 'B')
]

all_intra_stats = {}
all_inter_stats = {}

for channel_data, channel_name in channels:
    print(f"\n{'='*60}")
    print(f"ANALIZA KANAŁU {channel_name} - BloodMNIST 128x128")
    print(f"{'='*60}")

    # Analiza wewnątrzklasowa
    intra_stats = calculate_intra_class_distances(channel_data, labels, channel_name)
    all_intra_stats[channel_name] = intra_stats

    # Analiza międzyklasowa
    inter_stats = calculate_inter_class_distances(channel_data, labels, channel_name)
    all_inter_stats[channel_name] = inter_stats

# Podsumowanie
print(f"\n{'='*60}")
print("PODSUMOWANIE - BloodMNIST")
print(f"{'='*60}")
print(f"Rozdzielczość obrazów: {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"Próbek na klasę: {SAMPLES_PER_CLASS}")
print(f"Całkowita liczba próbek: {len(images)}")

for channel_name in ['R', 'G', 'B']:
    print(f"\nKanał {channel_name}:")

    # Średnia odległość wewnątrzklasowa
    intra_means = [stats['mean_distance'] for stats in all_intra_stats[channel_name].values()]
    if intra_means:
        print(f"  Średnia odległość wewnątrzklasowa: {np.mean(intra_means):.2f} ± {np.std(intra_means):.2f}")
        print(f"  Zakres odległości wewnątrzklasowych: {np.min(intra_means):.2f} - {np.max(intra_means):.2f}")

    # Średnia odległość międzyklasowa
    inter_means = [stats['mean_distance'] for stats in all_inter_stats[channel_name].values()]
    if inter_means:
        print(f"  Średnia odległość międzyklasowa: {np.mean(inter_means):.2f} ± {np.std(inter_means):.2f}")
        print(f"  Zakres odległości międzyklasowych: {np.min(inter_means):.2f} - {np.max(inter_means):.2f}")

print(f"\nAnaliza BloodMNIST zakończona!")
print(f"Przeanalizowano {len(images)} obrazów ({SAMPLES_PER_CLASS} na klasę) w rozdzielczości {IMAGE_SIZE}x{IMAGE_SIZE}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

  # Wizualizacja wyników
def plot_distance_comparison1():
  # Set high-quality scientific style
  plt.rcParams.update({
      'font.size': 12,
      'font.family': 'serif',
      'font.serif': ['Times New Roman', 'Computer Modern Roman'],
      'axes.linewidth': 1.2,
      'axes.spines.left': True,
      'axes.spines.bottom': True,
      'axes.spines.top': False,
      'axes.spines.right': False,
      'xtick.major.size': 6,
      'xtick.minor.size': 3,
      'ytick.major.size': 6,
      'ytick.minor.size': 3,
      'legend.frameon': True,
      'legend.fancybox': False,
      'legend.edgecolor': 'black',
      'legend.facecolor': 'white',
      'legend.framealpha': 0.9,
      'figure.dpi': 300,
      'savefig.dpi': 300,
      'savefig.bbox': 'tight',
      'savefig.pad_inches': 0.1
  })

  # Set seaborn style for scientific publications
  sns.set_style("whitegrid", {
      'axes.grid': True,
      'axes.edgecolor': 'black',
      'grid.color': 'gray',
      'grid.alpha': 0.4,
      'grid.linewidth': 0.8
  })

  # Set color palette - using colorblind-friendly colors
  colors = ['red', 'green', 'blue']

  # Create figure with higher DPI and better proportions
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6), dpi=300)
  ax1.set_ylim(15, 95)
  ax2.set_ylim(15, 95)

  channels = ['R', 'G', 'B']

  # Prepare data for seaborn (convert to long format)
  # Intra-class distances
  intra_df_list = []
  for i, channel in enumerate(channels):
      means = [stats['mean_distance'] for stats in all_intra_stats[channel].values()]
      for mean_val in means:
          intra_df_list.append({'Channel': channel, 'Distance': mean_val, 'Type': 'Intra-class'})

  intra_df = pd.DataFrame(intra_df_list)

  # Inter-class distances
  inter_df_list = []
  for i, channel in enumerate(channels):
      means = [stats['mean_distance'] for stats in all_inter_stats[channel].values()]
      for mean_val in means:
          inter_df_list.append({'Channel': channel, 'Distance': mean_val, 'Type': 'Inter-class'})

  inter_df = pd.DataFrame(inter_df_list)

  # Plot 1: Intra-class distances with seaborn
  sns.boxplot(
    data=intra_df,
    x='Channel',
    y='Distance',
    hue='Channel',
    ax=ax1,
    palette=colors,
    linewidth=1.2,
    fliersize=4,
    legend=False)
  ax1.set_title('Intra-class Distances - BloodMNIST', fontsize=14, pad=10)
  ax1.set_ylabel('Euclidean Distance', fontsize=12, fontweight='bold')
  ax1.set_xlabel('Color Channel', fontsize=12, fontweight='bold')
  ax1.tick_params(axis='both', which='major', labelsize=10)

  # Add statistical annotations (mean values)
  for i, channel in enumerate(channels):
      means = [stats['mean_distance'] for stats in all_intra_stats[channel].values()]
      mean_val = np.mean(means)
      ax1.text(i, ax1.get_ylim()[1] * 0.95, f'mean = {mean_val:.1f}',
              ha='center', va='top', fontsize=9, fontweight='bold',
              bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

  # Plot 2: Inter-class distances with seaborn
  sns.boxplot(
    data=inter_df,
    x='Channel',
    y='Distance',
    hue='Channel',
    ax=ax2,
    palette=colors,
    linewidth=1.2,
    fliersize=4,
    legend=False)
  ax2.set_title('Inter-class Distances - BloodMNIST', fontsize=14, pad=10)
  ax2.set_ylabel('Euclidean Distance', fontsize=12, fontweight='bold')
  ax2.set_xlabel('Color Channel', fontsize=12, fontweight='bold')
  ax2.tick_params(axis='both', which='major', labelsize=10)

  # Add statistical annotations (mean values)
  for i, channel in enumerate(channels):
      means = [stats['mean_distance'] for stats in all_inter_stats[channel].values()]
      mean_val = np.mean(means)
      ax2.text(i, ax2.get_ylim()[1] * 0.95, f'mean = {mean_val:.1f}',
              ha='center', va='top', fontsize=9, fontweight='bold',
              bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

  # Improve layout
  plt.tight_layout(rect=[0, 0.03, 1, 0.93])

  # Add subtle background color for better contrast
  fig.patch.set_facecolor('white')

  # Optional: Add grid customization for better readability
  for ax in [ax1, ax2]:
      ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.8)
      ax.set_axisbelow(True)

  plt.show()

  # Uruchom wizualizację
plot_distance_comparison1()

In [ ]:
def plot_intra_class_spiderplot(all_intra_stats):
    import matplotlib.pyplot as plt
    import numpy as np

    channels = ['R', 'G', 'B']
    class_names = ['basophil', 'eosinophil', 'erythroblast', 'granulocyte',
                   'lymphocyte', 'monocyte', 'neutrophil', 'platelet']
    colors = ['#E53E3E', '#38A169', '#3182CE']  # R, G, B

    angles = np.linspace(0, 2 * np.pi, len(class_names), endpoint=False).tolist()
    angles += angles[:1]  # close loop

    plt.rcParams['font.family'] = 'sans-serif'

    fig, ax = plt.subplots(figsize=(10, 8), subplot_kw=dict(polar=True))

    for i, channel in enumerate(channels):
        stats = all_intra_stats[channel]
        means = [stats.get(cls_id, {'mean_distance': 0})['mean_distance'] for cls_id in range(len(class_names))]
        means += means[:1]  # close the loop

        ax.plot(angles, means, label=f'{channel}', color=colors[i], linewidth=2)
        ax.fill(angles, means, color=colors[i], alpha=0.15)

    # Custom label placement with offset and angle
    label_radius = ax.get_rmax() * 1.13  # Push labels out a bit beyond the outermost grid
    for angle, label in zip(angles[:-1], class_names):
        ax.text(angle, label_radius, label,
                fontsize=10, ha='center', va='center',
                rotation=25, rotation_mode='anchor')

    ax.set_title('Mean Intra-class Euclidean Distances - RGB Channel Analysis\nBloodMNIST',
                 pad=50, fontsize=14)

    ax.set_xticks(angles[:-1])  # Keep angles for radial lines
    ax.set_xticklabels([])  # But remove their labels
    ax.set_rlabel_position(0)
    ax.xaxis.grid(True, color='gray', linestyle='--', linewidth=0.5)  # Radial lines
    ax.yaxis.grid(True)  # Circular gridlines

    # Legend in bottom-right corner
    ax.legend(loc='lower right', bbox_to_anchor=(1.25, -0.1), fontsize=10)

    plt.tight_layout()
    plt.show()

plot_intra_class_spiderplot(all_intra_stats)